# Creates SBML Models for Sub-SBML
Note: SBML for Bistable Switch already created in Adding rR2 notebook. File name is BiSwitch_CRN.xml

1. Simple rR1 diffusion model (Agrima's atc model)
1. Advanced diffusion of rR1 (Agrima's dna export model)
1. External source with rI# (try integration)

# Bistable Switch
SBML file down below

In [1]:
from biocrnpyler import *
from genelet import *
import pylab as plt
import numpy as np

In [2]:
# Create CRN for bistable switch
Core1_off = Species ("Core1")
Core2_off = Species ("Core2")

Core1 = Genelet(Core1_off, transcript = "rR1_i", activator = "dA2", inhibitor = "rR2" )
Core2 = Genelet(Core2_off, transcript = "rR2", activator = "dA1", inhibitor = "rR1_i" )

M_BiSwitch = Mixture(name = "Switch_test", components = [Core1, Core2], parameter_file = "default_parameters.txt")
repr(M_BiSwitch)
CRN_BiSwitch = M_BiSwitch.compile_crn()

# Write SBML file
CRN_BiSwitch.write_sbml_file('BiSwitch_CRN.xml')

#print(CRN_BiSwitch)

True

# Produce 2N
SBML file below with DFHBI in separate reaction

###### Create CRN of Produce 2N
Prod2_off = Species("Prod2")
Broccoli_Aptamer = Species ("BrocApt")

Prod2 = Genelet(Prod2_off, transcript= Broccoli_Aptamer , activator= "dA1"  , inhibitor= "rR1", 
                rnap="RNAP", rnaseH="RNAseH")
M_Prod2 = Mixture(name = "Produce2_test", components = [Prod2], parameter_file = "default_parameters.txt")
repr(M_Prod2)
CRN_Prod2 = M_Prod2.compile_crn()

rxn1 = Reaction([Broccoli_Aptamer,DFHBI], [ComplexSpecies([Broccoli_Aptamer, DFHBI], name="Flu1")], k = 9.96e-2)
#CRN_Prod2.add_reactions([rxn1])

# Write SBML file
#CRN_Prod2.write_sbml_file('Prod2_CRN.xml')

#print(CRN_Prod2.pretty_print())

#### Create CRN for DFHBI and Broc Apt Binding

### Create species that do RNA transport ###

SID_f = Species ("SID_f") # free export protein
rR1 = Species ("rR1") # repressor for genelet 2
rR2 = Species ("rR2") # repressor for genelet 1
SID_b = Species ("SID_b") # bound to membrane


# rR1 external reservoir

In [3]:
##### Creating rR1 reservoir model####

from subsbml import createBasicSubsystem, SimpleModel


rR1_ss, rR1_model = createBasicSubsystem('external_reservoir')

per_second = rR1_model.createNewUnitDefinition(uId = 'per_second',uKind = libsbml.UNIT_KIND_SECOND,
                                                uExponent = -1)
substance = rR1_model.createNewUnitDefinition('substance',libsbml.UNIT_KIND_DIMENSIONLESS, uExponent = 1)

rR1_model.createNewSpecies('rna_rR1_e',sComp = 'external_reservoir', ListOfAmounts = 10, 
                             sConstant = False, sSubstance = 'substance')

# Write to SBML file 
exsub = rR1_ss.writeSBML('rR1_external_reservoir.xml')
print(rR1_ss)

# rR1 simple membrane model

rR1_internal $\leftrightarrow$ rR1_external

In [8]:
##### Creating a basic membrane model for IPTG transport####

from subsbml import createNewSubsystem, SimpleModel, SimpleReaction
import libsbml

rR1_mb_ss = createNewSubsystem()
model = rR1_mb_ss.createNewModel('rR1_membrane','second','mole','substance')

rR1_mb = SimpleModel(model)

per_second = rR1_mb.createNewUnitDefinition('per_second',libsbml.UNIT_KIND_SECOND,-1,0,1)
substance = rR1_mb.createNewUnitDefinition('substance',libsbml.UNIT_KIND_DIMENSIONLESS, 1, 0, 1)

################# Two important lines of code when creating membrane #########################
rR1_mb.createNewCompartment('internal','internal',1e-6,'litre',True)
rR1_mb.createNewCompartment('external','external',1e-6,'litre',True)
##############################################################################################

rR1_mb.createNewSpecies( 'rna_rR1_e','external',0,False,'substance')
rR1_mb.createNewSpecies( 'rna_rR1_i','internal',0,False,'substance')

rR1_mb.createNewParameter( 'kb',2e-3,False,'per_second')
rR1_mb.createNewParameter( 'kd',1e-5,False,'per_second')

# model = IPTG_mb_ss.getSBMLDocument().getModel()
r1_sbml = model.createReaction()
r1 = SimpleReaction(r1_sbml)
r1.setId('r1')
r1.setReversible(True)
r1.createNewReactant('rna_rR1_e',False,1)
r1.createNewProduct('rna_rR1_i', False, 1)
r1.createRate('kb * rna_rR1_e - kd * rna_rR1_i')

# Write to SBML file 
_ = rR1_mb_ss.writeSBML('simplemembrane_rR1.xml')
print('Model written to a SBML file successfully')

Model written to a SBML file successfully


# rR1 advanced membrane model (SID)

In [5]:
##### Creating a advanced membrane model for RNA transport####

from subsbml import createNewSubsystem, SimpleModel, SimpleReaction

rnaexp_mb_ss = createNewSubsystem()
model = rnaexp_mb_ss.createNewModel('rnaexp_membrane','second','mole','substance')

rnaexp_mb = SimpleModel(model)

per_second = rnaexp_mb.createNewUnitDefinition('per_second',libsbml.UNIT_KIND_SECOND,-1,0,1)
substance = rnaexp_mb.createNewUnitDefinition('substance',libsbml.UNIT_KIND_DIMENSIONLESS, 1, 0, 1)

################# Two important lines of code when creating membrane #########################
rnaexp_mb.createNewCompartment('internal','internal',1e-6,'litre',True)
rnaexp_mb.createNewCompartment('external','external',1e-6,'litre',True)
##############################################################################################

rnaexp_mb.createNewSpecies( 'rna_rR1_e','external',0,False,'substance')
rnaexp_mb.createNewSpecies( 'rna_rR1_i','internal',0,False,'substance')
rnaexp_mb.createNewSpecies( 'VirE2_b','internal',1,False,'substance')
rnaexp_mb.createNewSpecies( 'VirE2_b_rna_rR1_i','internal',0,False,'substance')

rnaexp_mb.createNewParameter( 'kb1',0.002,False,'per_second')
rnaexp_mb.createNewParameter( 'kd1',0.00001,False,'per_second')
rnaexp_mb.createNewParameter( 'kd2',0.0002,False,'per_second')


model = rnaexp_mb_ss.getSBMLDocument().getModel()

R1 = SimpleReaction(model.createReaction())
R1.setId('R1')
R1.setReversible(True)
R1.createNewReactant('VirE2_b',False,1)
R1.createNewReactant('rna_rR1_i', False, 1)
R1.createNewProduct('VirE2_b_rna_rR1_i', False, 1)
R1.createRate('kb1 * VirE2_b * rna_rR1_i - kd1 * VirE2_b_rna_rR1_i')


R2 = SimpleReaction(model.createReaction())
R2.setId('R2')
R2.setReversible(False)
R2.createNewReactant('VirE2_b_rna_rR1_i',False,1)
R2.createNewProduct('rna_rR1_e',False,1)
R2.createNewProduct('VirE2_b',False,1)
R2.createRate('kd2 * VirE2_b_rna_rR1_i')


# Write to SBML file 
_ = rnaexp_mb_ss.writeSBML('RNAexp_membrane1.xml')
print('Model written to a SBML file successfully')

Model written to a SBML file successfully


In [6]:
##### Creating a advanced membrane model for DNA transport####

from subsbml import createNewSubsystem, SimpleModel, SimpleReaction
import libsbml

rnaexp_mb_ss = createNewSubsystem()
model = rnaexp_mb_ss.createNewModel('rnaexp_membrane','second','mole','substance')

rnaexp_mb = SimpleModel(model)

per_second = rnaexp_mb.createNewUnitDefinition('per_second',libsbml.UNIT_KIND_SECOND,-1,0,1)
substance = rnaexp_mb.createNewUnitDefinition('substance',libsbml.UNIT_KIND_DIMENSIONLESS, 1, 0, 1)

################# Two important lines of code when creating membrane #########################
rnaexp_mb.createNewCompartment('internal','internal',1e-6,'litre',True)
rnaexp_mb.createNewCompartment('external','external',1e-6,'litre',True)
##############################################################################################

rnaexp_mb.createNewSpecies( 'rna_rR1_e','external',0,False,'substance')
rnaexp_mb.createNewSpecies( 'rna_rR1_i','internal',0,False,'substance')
rnaexp_mb.createNewSpecies( 'VirE2_b','internal',1,False,'substance')
rnaexp_mb.createNewSpecies( 'VirE2_b_rna_rR1_i','internal',0,False,'substance')

rnaexp_mb.createNewParameter( 'kb',0.2,False,'per_second')
rnaexp_mb.createNewParameter( 'kd',0.01,False,'per_second')

model = rnaexp_mb_ss.getSBMLDocument().getModel()

R1 = SimpleReaction(model.createReaction())
R1.setId('R1')
R1.setReversible(True)
R1.createNewReactant('VirE2_b',False,1)
R1.createNewReactant('rna_rR1_i', False, 1)
R1.createNewProduct('VirE2_b_rna_rR1_i', False, 1)
R1.createRate('kb * VirE2_b * rna_rR1_i - kd * VirE2_b_rna_rR1_i')


R2 = SimpleReaction(model.createReaction())
R2.setId('R2')
R2.setReversible(True)
R2.createNewReactant('VirE2_b_rna_rR1_i',False,1)
R2.createNewProduct('rna_rR1_e',False,1)
R2.createNewProduct('VirE2_b',False,1)
R2.createRate('kd * VirE2_b_rna_rR1_i- kb * VirE2_b * rna_rR1_e')


# Write to SBML file 
_ = rnaexp_mb_ss.writeSBML('RNAexp_membrane3.xml')
print('Model written to a SBML file successfully')

Model written to a SBML file successfully
